<a href="https://colab.research.google.com/github/ustab/tensorf-ml/blob/main/data_cleaning_challenge_handling_missing_values.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### All days of the challange:

* [Day 1: Handling missing values](https://www.kaggle.com/rtatman/data-cleaning-challenge-handling-missing-values)
* [Day 2: Scaling and normalization](https://www.kaggle.com/rtatman/data-cleaning-challenge-scale-and-normalize-data)
* [Day 3: Parsing dates](https://www.kaggle.com/rtatman/data-cleaning-challenge-parsing-dates/)
* [Day 4: Character encodings](https://www.kaggle.com/rtatman/data-cleaning-challenge-character-encodings/)
* [Day 5: Inconsistent Data Entry](https://www.kaggle.com/rtatman/data-cleaning-challenge-inconsistent-data-entry/)
___
Welcome to day 1 of the 5-Day Data Challenge! Today, we're going to be looking at how to deal with missing values. To get started, click the blue "Fork Notebook" button in the upper, right hand corner. This will create a private copy of this notebook that you can edit and play with. Once you're finished with the exercises, you can choose to make your notebook public to share with others. :)

> **Your turn!** As we work through this notebook, you'll see some notebook cells (a block of either code or text) that has "Your Turn!" written in it. These are exercises for you to do to help cement your understanding of the concepts we're talking about. Once you've written the code to answer a specific question, you can run the code by clicking inside the cell (box with code in it) with the code you want to run and then hit CTRL + ENTER (CMD + ENTER on a Mac). You can also click in a cell and then click on the right "play" arrow to the left of the code. If you want to run all the code in your notebook, you can use the double, "fast forward" arrows at the bottom of the notebook editor.

Here's what we're going to do today:

* [Take a first look at the data](#Take-a-first-look-at-the-data)
* [See how many missing data points we have](#See-how-many-missing-data-points-we-have)
* [Figure out why the data is missing](#Figure-out-why-the-data-is-missing)
* [Drop missing values](#Drop-missing-values)
* [Filling in missing values](#Filling-in-missing-values)

Let's get started!

# Take a first look at the data
________

The first thing we'll need to do is load in the libraries and datasets we'll be using. For today, I'll be using a dataset of events that occured in American Football games for demonstration, and you'll be using a dataset of building permits issued in San Francisco.

> **Important!** Make sure you run this cell yourself or the rest of your code won't work!

In [3]:
# modules we'll use
import pandas as pd
import numpy as np

# read in all our data
nfl_data = pd.read_csv("/content/NFL Play by Play 2009-2017 (v4).csv")

# set seed for reproducibility
np.random.seed(0) 

In [4]:
sf_permits = pd.read_excel("/content/DataDictionaryBuildingPermit.xlsx")

The first thing I do when I get a new dataset is take a look at some of it. This lets me see that it all read in correctly and get an idea of what's going on with the data. In this case, I'm looking to see if I see any missing values, which will be reprsented with `NaN` or `None`.

In [9]:
# look at a few rows of the nfl_data file. I can see a handful of missing data already!
nfl_data.sample(15)

,Date,GameID,Drive,qtr,down,time,TimeUnder,TimeSecs,PlayTimeDiff,SideofField,...,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season
19505,2009-11-01,2009110109,9,2,2.0,04:55,5,2095.0,35.0,TEN,...,NaN,0.730561,0.269439,0.717599,0.282401,0.730561,-0.012962,NaN,NaN,2009.0
13856,2009-10-18,2009101803,19,4,NaN,10:08,11,608.0,4.0,BAL,...,NaN,0.951397,0.048603,0.951269,0.048731,0.951397,-0.000128,NaN,NaN,2009.0
14300,2009-10-18,2009101806,6,2,NaN,11:48,12,2508.0,0.0,CAR,...,NaN,0.490459,0.509541,0.490822,0.509178,0.490459,0.000364,NaN,NaN,2009.0
1686,2009-09-13,2009091300,21,4,1.0,07:28,8,448.0,7.0,MIA,...,0.425303,0.973178,0.026822,0.976798,0.023202,0.026822,-0.003620,-0.006008,0.002388,2009.0
1384,2009-09-13,2009091301,2,1,4.0,10:23,11,3323.0,6.0,KC,...,NaN,0.530714,0.469286,0.575536,0.424464,0.530714,0.044822,NaN,NaN,2009.0
8864,2009-10-04,2009100402,25,4,1.0,13:41,14,821.0,41.0,HOU,...,-1.263093,0.967827,0.032173,0.970209,0.029791,0.032173,-0.002382,0.005641,-0.008023,2009.0
9285,2009-10-04,2009100405,4,1,3.0,07:40,8,3160.0,45.0,NYG,...,NaN,0.235411,0.764589,0.395569,0.604431,0.764589,-0.160158,NaN,NaN,2009.0
13681,2009-10-18,2009101802,20,4,1.0,04:31,5,271.0,5.0,JAC,...,-0.297557,0.378037,0.621963,0.351970,0.648030,0.378037,-0.026067,-0.010130,-0.015937,2009.0
9621,2009-10-04,2009100406,19,4,1.0,01:38,2,98.0,22.0,NE,...,-0.189526,0.833907,0.166093,0.860234,0.139766,0.166093,-0.026327,-0.015891,-0.010436,2009.0
3822,2009-09-20,2009092004,4,1,1.0,07:43,8,3163.0,36.0,OAK,...,NaN,0.575902,0.424098,0.573886,0.426114,0.575902,-0.002016,NaN,NaN,2009.0


Yep, it looks like there's some missing values. What about in the sf_permits dataset?

In [6]:
# your turn! Look at a couple of rows from the sf_permits dataset. Do you notice any missing data?

# your code goes here :)
sf_permits

,Sl No,Column name,Description
0,1.0,Permit Number,Number assigned while filing
1,2.0,Permit Type,Type of the permit represented numerically.
2,3.0,Permit Type Definition,"Description of the Permit type, for example\n ..."
3,4.0,Permit Creation Date,"Date on which permit created, later than \nor ..."
4,5.0,Block,Related to address
5,6.0,Lot,Related to address
6,7.0,Street Number,Related to address
7,8.0,Street Number Suffix,Related to address
8,9.0,Street Name,Related to address
9,10.0,Street Name Suffix,Related to address


# See how many missing data points we have
___

Ok, now we know that we do have some missing values. Let's see how many we have in each column. 

In [8]:
nfl_data.shape

(21814, 102)

In [12]:
missing_values_count = nfl_data.isnull().sum()
missing_values_count

Date            0
GameID          0
Drive           0
qtr             0
down         3360
            ...  
Win_Prob     1432
WPA           332
airWPA      13541
yacWPA      13557
Season          1
Length: 102, dtype: int64

In [15]:
# get the number of missing data points per column
missing_values_count = nfl_data.isnull().sum()

# look at the # of missing points in the first ten columns
missing_values_count[0:10]

Date               0
GameID             0
Drive              0
qtr                0
down            3360
time              17
TimeUnder          0
TimeSecs          17
PlayTimeDiff      34
SideofField       31
dtype: int64

That seems like a lot! It might be helpful to see what percentage of the values in our dataset were missing to give us a better sense of the scale of this problem:

In [18]:
# how many total missing values do we have?
total_cells = np.product(nfl_data.shape)
total_missing = missing_values_count.sum()

# percent of data that is missing
(total_missing/total_cells) * 100

24.98143843583092

Wow, almost a quarter of the cells in this dataset are empty! In the next step, we're going to take a closer look at some of the columns with missing values and try to figure out what might be going on with them.

In [31]:
# your turn! Find out what percent of the sf_permits dataset is missing
# how many total missing values do we have?
total_missing=sf_permits.isnull().sum().sum()
percentage_missing=total_missing/np.product(sf_permits.shape)*100
percentage_missing

0.0

# Figure out why the data is missing
____
 
This is the point at which we get into the part of data science that I like to call "data intution", by which I mean "really looking at your data and trying to figure out why it is the way it is and how that will affect your analysis". It can be a frustrating part of data science, especially if you're newer to the field and don't have a lot of experience. For dealing with missing values, you'll need to use your intution to figure out why the value is missing. One of the most important question you can ask yourself to help figure this out is this:

> **Is this value missing becuase it wasn't recorded or becuase it dosen't exist?**

If a value is missing becuase it doens't exist (like the height of the oldest child of someone who doesn't have any children) then it doesn't make sense to try and guess what it might be. These values you probalby do want to keep as NaN. On the other hand, if a value is missing becuase it wasn't recorded, then you can try to guess what it might have been based on the other values in that column and row. (This is called "imputation" and we'll learn how to do it next! :)

Let's work through an example. Looking at the number of missing values in the nfl_data dataframe, I notice that the column `TimesSec` has a lot of missing values in it: 

In [32]:
# look at the # of missing points in the first ten columns
missing_values_count[0:10]

Date               0
GameID             0
Drive              0
qtr                0
down            3360
time              17
TimeUnder          0
TimeSecs          17
PlayTimeDiff      34
SideofField       31
dtype: int64

By looking at [the documentation](https://www.kaggle.com/maxhorowitz/nflplaybyplay2009to2016), I can see that this column has information on the number of seconds left in the game when the play was made. This means that these values are probably missing because they were not recorded, rather than because they don't exist. So, it would make sense for us to try and guess what they should be rather than just leaving them as NA's.

On the other hand, there are other fields, like `PenalizedTeam` that also have lot of missing fields. In this case, though, the field is missing because if there was no penalty then it doesn't make sense to say *which* team was penalized. For this column, it would make more sense to either leave it empty or to add a third value like "neither" and use that to replace the NA's.

> **Tip:** This is a great place to read over the dataset documentation if you haven't already! If you're working with a dataset that you've gotten from another person, you can also try reaching out to them to get more information.

If you're doing very careful data analysis, this is the point at which you'd look at each column individually to figure out the best strategy for filling those missing values. For the rest of this notebook, we'll cover some "quick and dirty" techniques that can help you with missing values but will probably also end up removing some useful information or adding some noise to your data.

## Your turn!

* Look at the columns `Street Number Suffix` and `Zipcode` from the `sf_permits` datasets. Both of these contain missing values. Which, if either, of these are missing because they don't exist? Which, if either, are missing because they weren't recorded?

# Drop missing values
___

If you're in a hurry or don't have a reason to figure out why your values are missing, one option you have is to just remove any rows or columns that contain missing values. (Note: I don't generally recommend this approch for important projects! It's usually worth it to take the time to go through your data and really look at all the columns with missing values one-by-one to really get to know your dataset.)  

If you're sure you want to drop rows with missing values, pandas does have a handy function, `dropna()` to help you do this. Let's try it out on our NFL dataset!

In [33]:
# remove all the rows that contain a missing value
nfl_data.dropna()

,Date,GameID,Drive,qtr,down,time,TimeUnder,TimeSecs,PlayTimeDiff,SideofField,...,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season


Oh dear, it looks like that's removed all our data! 😱 This is because every row in our dataset had at least one missing value. We might have better luck removing all the *columns* that have at least one missing value instead.

In [34]:
# remove all columns with at least one missing value
columns_with_na_dropped = nfl_data.dropna(axis=1)
columns_with_na_dropped.head()

,Date,GameID,Drive,qtr,TimeUnder,ydstogo,ydsnet,desc,PlayAttempted,Yards.Gained,...,Onsidekick,PlayType,Passer_ID,PassAttempt,AirYards,YardsAfterCatch,QBHit,InterceptionThrown,Rusher_ID,RushAttempt
0,2009-09-10,2009091000,1,1,15,0,0,R.Bironas kicks 67 yards from TEN 30 to PIT 3....,1,39,...,0,Kickoff,None,0,0,0,0,0,None,0
1,2009-09-10,2009091000,1,1,15,10,5,(14:53) B.Roethlisberger pass short left to H....,1,5,...,0,Pass,00-0022924,1,-3,8,0,0,None,0
2,2009-09-10,2009091000,1,1,15,5,2,(14:16) W.Parker right end to PIT 44 for -3 ya...,1,-3,...,0,Run,None,0,0,0,0,0,00-0022250,1
3,2009-09-10,2009091000,1,1,14,8,2,(13:35) (Shotgun) B.Roethlisberger pass incomp...,1,0,...,0,Pass,00-0022924,1,34,0,0,0,None,0
4,2009-09-10,2009091000,1,1,14,8,2,(13:27) (Punt formation) D.Sepulveda punts 54 ...,1,0,...,0,Punt,None,0,0,0,0,0,None,0


In [36]:
# just how much data did we lose?
print("Columns in original dataset: %d \n" % nfl_data.shape[1])

Columns in original dataset: 102 



In [37]:
print("Columns with na's dropped: %d" % columns_with_na_dropped.shape[1])

Columns with na's dropped: 23


We've lost quite a bit of data, but at this point we have successfully removed all the `NaN`'s from our data. 

In [39]:
# Your turn! Try removing all the rows from the sf_permits dataset that contain missing values. How many are left?
# remove all columns with at least one missing value
columns_with_na_dropped = sf_permits.dropna(axis=1)
columns_with_na_dropped.head()

,Sl No,Column name,Description
0,1.0,Permit Number,Number assigned while filing
1,2.0,Permit Type,Type of the permit represented numerically.
2,3.0,Permit Type Definition,"Description of the Permit type, for example\n ..."
3,4.0,Permit Creation Date,"Date on which permit created, later than \nor ..."
4,5.0,Block,Related to address


In [40]:
# Now try removing all the columns with empty values. Now how much of your data is left?
print("Columns with na's dropped: %d" % columns_with_na_dropped.shape[1])

Columns with na's dropped: 3


# Filling in missing values automatically
_____

Another option is to try and fill in the missing values. For this next bit, I'm getting a small sub-section of the NFL data so that it will print well.

In [41]:
# get a small subset of the NFL dataset
subset_nfl_data = nfl_data.loc[:, 'EPA':'Season'].head()
subset_nfl_data

,EPA,airEPA,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season
0,2.014474,NaN,NaN,0.485675,0.514325,0.546433,0.453567,0.485675,0.060758,NaN,NaN,2009.0
1,0.077907,-1.068169,1.146076,0.546433,0.453567,0.551088,0.448912,0.546433,0.004655,-0.032244,0.036899,2009.0
2,-1.402760,NaN,NaN,0.551088,0.448912,0.510793,0.489207,0.551088,-0.040295,NaN,NaN,2009.0
3,-1.712583,3.318841,-5.031425,0.510793,0.489207,0.461217,0.538783,0.510793,-0.049576,0.106663,-0.156239,2009.0
4,2.097796,NaN,NaN,0.461217,0.538783,0.558929,0.441071,0.461217,0.097712,NaN,NaN,2009.0


We can use the Panda's fillna() function to fill in missing values in a dataframe for us. One option we have is to specify what we want the `NaN` values to be replaced with. Here, I'm saying that I would like to replace all the `NaN` values with 0.

In [42]:
# replace all NA's with 0
subset_nfl_data.fillna(0)

,EPA,airEPA,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season
0,2.014474,0.000000,0.000000,0.485675,0.514325,0.546433,0.453567,0.485675,0.060758,0.000000,0.000000,2009.0
1,0.077907,-1.068169,1.146076,0.546433,0.453567,0.551088,0.448912,0.546433,0.004655,-0.032244,0.036899,2009.0
2,-1.402760,0.000000,0.000000,0.551088,0.448912,0.510793,0.489207,0.551088,-0.040295,0.000000,0.000000,2009.0
3,-1.712583,3.318841,-5.031425,0.510793,0.489207,0.461217,0.538783,0.510793,-0.049576,0.106663,-0.156239,2009.0
4,2.097796,0.000000,0.000000,0.461217,0.538783,0.558929,0.441071,0.461217,0.097712,0.000000,0.000000,2009.0


I could also be a bit more savvy and replace missing values with whatever value comes directly after it in the same column. (This makes a lot of sense for datasets where the observations have some sort of logical order to them.)

In [43]:
# replace all NA's the value that comes directly after it in the same column, 
# then replace all the reamining na's with 0
subset_nfl_data.fillna(method = 'bfill', axis=0).fillna(0)

,EPA,airEPA,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season
0,2.014474,-1.068169,1.146076,0.485675,0.514325,0.546433,0.453567,0.485675,0.060758,-0.032244,0.036899,2009.0
1,0.077907,-1.068169,1.146076,0.546433,0.453567,0.551088,0.448912,0.546433,0.004655,-0.032244,0.036899,2009.0
2,-1.402760,3.318841,-5.031425,0.551088,0.448912,0.510793,0.489207,0.551088,-0.040295,0.106663,-0.156239,2009.0
3,-1.712583,3.318841,-5.031425,0.510793,0.489207,0.461217,0.538783,0.510793,-0.049576,0.106663,-0.156239,2009.0
4,2.097796,0.000000,0.000000,0.461217,0.538783,0.558929,0.441071,0.461217,0.097712,0.000000,0.000000,2009.0


Filling in missing values is also known as "imputation", and you can find more exercises on it [in this lesson, also linked under the "More practice!" section](https://www.kaggle.com/dansbecker/handling-missing-values). First, however, why don't you try replacing some of the missing values in the sf_permit dataset?

In [52]:
sf_permits.fillna(method = 'bfill', axis=0)

,Sl No,Column name,Description
0,1.0,Permit Number,Number assigned while filing
1,2.0,Permit Type,Type of the permit represented numerically.
2,3.0,Permit Type Definition,"Description of the Permit type, for example\n ..."
3,4.0,Permit Creation Date,"Date on which permit created, later than \nor ..."
4,5.0,Block,Related to address
5,6.0,Lot,Related to address
6,7.0,Street Number,Related to address
7,8.0,Street Number Suffix,Related to address
8,9.0,Street Name,Related to address
9,10.0,Street Name Suffix,Related to address


In [51]:
sf_permits.fillna(0, axis=0)

,Sl No,Column name,Description
0,1.0,Permit Number,Number assigned while filing
1,2.0,Permit Type,Type of the permit represented numerically.
2,3.0,Permit Type Definition,"Description of the Permit type, for example\n ..."
3,4.0,Permit Creation Date,"Date on which permit created, later than \nor ..."
4,5.0,Block,Related to address
5,6.0,Lot,Related to address
6,7.0,Street Number,Related to address
7,8.0,Street Number Suffix,Related to address
8,9.0,Street Name,Related to address
9,10.0,Street Name Suffix,Related to address


In [48]:
# Your turn! Try replacing all the NaN's in the sf_permits data with the one that
# comes directly after it and then replacing any remaining NaN's with 0
sf_permits.fillna(method = 'bfill', axis=0).fillna(0)

,Sl No,Column name,Description
0,1.0,Permit Number,Number assigned while filing
1,2.0,Permit Type,Type of the permit represented numerically.
2,3.0,Permit Type Definition,"Description of the Permit type, for example\n ..."
3,4.0,Permit Creation Date,"Date on which permit created, later than \nor ..."
4,5.0,Block,Related to address
5,6.0,Lot,Related to address
6,7.0,Street Number,Related to address
7,8.0,Street Number Suffix,Related to address
8,9.0,Street Name,Related to address
9,10.0,Street Name Suffix,Related to address


And that's it for today! If you have any questions, be sure to post them in the comments below or [on the forums](https://www.kaggle.com/questions-and-answers). 

Remember that your notebook is private by default, and in order to share it with other people or ask for help with it, you'll need to make it public. First, you'll need to save a version of your notebook that shows your current work by hitting the "Commit & Run" button. (Your work is saved automatically, but versioning your work lets you go back and look at what it was like at the point you saved it. It also let's you share a nice compiled notebook instead of just the raw code.) Then, once your notebook is finished running, you can go to the Settings tab in the panel to the left (you may have to expand it by hitting the [<] button next to the "Commit & Run" button) and setting the "Visibility" dropdown to "Public".

# More practice!
___

If you're looking for more practice handling missing values, check out these extra-credit\* exercises:

* [Handling Missing Values](https://www.kaggle.com/dansbecker/handling-missing-values): In this notebook Dan shows you several approaches to imputing missing data using scikit-learn's imputer. 
* Look back at the `Zipcode` column in the `sf_permits` dataset, which has some missing values. How would you go about figuring out what the actual zipcode of each address should be? (You might try using another dataset. You can search for datasets about San Fransisco on the [Datasets listing](https://www.kaggle.com/datasets).) 

\* no actual credit is given for completing the challenge, you just learn how to clean data real good :P